In [2]:
# Install required python packages
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [ ]:
# Import required packages
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2

# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "<your-client-id>"
CLIENT_SECRET = "<your-client-secret>"
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print("Go to the following link in your browser: " + authorize_url)
auth_code = input("Enter your Authorization Code here:")
credentials = flow.step2_exchange(auth_code)
http = httplib2.Http()
creds = credentials.authorize(http)
webmasters_service = build('searchconsole', 'v1', http=creds)

# How to get a list of Sites we have in GSC Account?

In [ ]:
# Get a list of site in my Google Search Console Account
site_list = webmasters_service.sites().list().execute()

site_list

# How to get Google Search Console Analytics Data using API?

In [ ]:
# the website we want to get the data for
website = "https://www.fishersci.com/"

# build a request body
request_body = {
    "startDate"	: '2024-03-01',
    "endDate" : '2024-04-01',
    "dimensions" : ['QUERY', 'PAGE'],
      "rowLimit" : 25000,
    "dataState" : "final"
}

# get the response using request body
response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()
len(response_data['rows'])

In [ ]:
# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow % 25000 == 0):
    # build a request body
    request_body = {
        "startDate": '2024-10-01',
        "endDate": '2024-10-31',
        "dimensions": ['QUERY', 'PAGE', 'COUNTRY', 'DEVICE'],  # Added country, device, and search appearance
        "rowLimit": 25000,
        "dataState": "final",
        'startRow' : startRow,
        "searchType": "web"  # Added search type for web search (you can change this to image, video, or news)
    }

    # get GSC response
    response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

    # update the rows
    startRow = startRow + len(response_data['rows'])
    print("fetched up to " + str(startRow) + " rows of data")

    # for loop to save all the rows in all_responses
    for row in response_data['rows']:
        all_responses.append(row)


In [ ]:
len(all_responses)
all_responses[0]

In [ ]:
import pandas as pd

# Build a Dataframe using Pandas

In [ ]:
# empty list to build a dataframe
data_for_df = []

for each in all_responses:
    temp = []

    # query
    temp.append(each['keys'][0])  # First key: QUERY

    # page
    temp.append(each['keys'][1])  # Second key: PAGE

    # clicks
    temp.append(each['clicks'])  # Clicks

    # impressions
    temp.append(each['impressions'])  # Impressions

    # ctr
    temp.append(each['ctr'])  # CTR

    # position
    temp.append(each['position'])  # Position

    # country (3rd key if included in dimensions)
    temp.append(each['keys'][2])  # Third key: COUNTRY

    # device (4th key if included in dimensions)
    temp.append(each['keys'][3])  # Fourth key: DEVICE

    # search appearance (5th key if included in dimensions)
    # temp.append(each['keys'][4])  # Fifth key: SEARCH_APPEARANCE

    data_for_df.append(temp)

data_for_df[0]

In [ ]:
import pandas as pd
df = pd.DataFrame(data_for_df, columns=['query', 'page', 'clicks', 'impressions', 'ctr', 'position', 'country', 'device'])
df

In [ ]:
unique_df = df.drop_duplicates(subset='query')
unique_df

In [ ]:
df['data_date'] = pd.Timestamp.now().normalize()

In [ ]:
all_responses[0]

In [ ]:
df.info()

In [ ]:
import pandas as pd
from google.oauth2 import service_account
from pandas_gbq import to_gbq
cred = 'thermofigher-gen-ai-5255b69aa6e4.json'
# Path to your service account JSON key
service_account_file = cred

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(service_account_file)


# Project ID and table details
project_id = 'thermofigher-gen-ai'
table_id = 'searchconsole.gsc_data'

# Push the DataFrame to BigQuery
to_gbq(df, destination_table=table_id, project_id=project_id, if_exists='append', credentials=credentials)

print("Data pushed successfully to BigQuery!")


In [1]:
pip install gspread pandas gspread_dataframe google-cloud-translate

Note: you may need to restart the kernel to use updated packages.


In [100]:
pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp312-cp312-macosx_11_0_arm64.whl (1.8 MB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import datetime, tzinfo
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
from pandas.plotting import register_matplotlib_converters
from google import auth as google_auth
import warnings
import concurrent.futures

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from google.cloud import bigquery
from google.oauth2 import service_account

import json
from datetime import datetime
import ast
import time
import re
import pandas as pd
import pandas_gbq

In [9]:

project_id = "thermofigher-gen-ai"
bq_client = bigquery.Client.from_service_account_json(project = project_id,json_credentials_path=cred)

In [48]:
from openai import OpenAI

OPENAI_API_KEY = "<Your-openai-key>"
client = OpenAI(api_key=OPENAI_API_KEY)
def get_embedding(text):
    response = client.embeddings.create(
        input=[text.replace("\n", " ")],  # Note that input is now a list
        model="text-embedding-3-small"  # Use the latest embedding model
    )
    
    return np.array(response.data[0].embedding)

In [38]:

project_id = "thermofigher-gen-ai"
bq_client = bigquery.Client.from_service_account_json(project = project_id,json_credentials_path=cred)
query = """select * from thermofigher-gen-ai.searchconsole.filtered_gsc_data"""
df = bq_client.query(query).to_dataframe()
df

/Users/anubhav.srivastav/myenv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,query,page,clicks,impressions,ctr,position,country,device,data_date
0,fisher scientific,https://www.fishersci.com/us/en/home.html,19612,66676,0.294139,1.013767,usa,DESKTOP,2024-10-09 00:00:00+00:00
1,thermo fisher,https://www.thermofisher.com/us/en/home.html,12253,28707,0.451813,1.006450,usa,DESKTOP,2024-10-09 00:00:00+00:00
2,thermofisher,https://www.thermofisher.com/us/en/home.html,11486,28387,0.412552,1.001727,usa,DESKTOP,2024-10-09 00:00:00+00:00
3,thermo fisher scientific,https://www.thermofisher.com/us/en/home.html,8990,23329,0.403953,1.018940,usa,DESKTOP,2024-10-09 00:00:00+00:00
4,fisher,https://www.fishersci.com/us/en/home.html,8621,46835,0.184072,1.128744,usa,DESKTOP,2024-10-09 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...
75320,dissecting forceps,https://www.fishersci.com/shop/products/fisher...,1,251,0.003984,4.410359,usa,DESKTOP,2024-10-09 00:00:00+00:00
75321,ambion,https://www.fishersci.com/shop/products/ambion...,1,251,0.003984,8.733068,usa,MOBILE,2024-10-09 00:00:00+00:00
75322,periodoc table,https://www.fishersci.com/us/en/periodic-table...,1,252,0.003968,10.825397,usa,DESKTOP,2024-10-09 00:00:00+00:00
75323,hydrion test strips,https://www.fishersci.com/shop/products/hydrio...,1,252,0.003968,9.837302,usa,MOBILE,2024-10-09 00:00:00+00:00


In [39]:
df["processed"]= None

In [40]:
df['data_date'] = df['data_date'].astype(str)

In [42]:
import openai
import numpy as np
import pandas as pd
from google.cloud import bigquery

# Convert NumPy array to list of floats
def embedding_to_list(embedding):
    return embedding.tolist()

# Define BigQuery table ID
table_id = 'thermofigher-gen-ai.searchconsole.filtered_gsc_data_embedding'

# Process and insert data in batches of 10 rows
batch_size = 200

for start in range(0, len(df), batch_size):
    end = start + batch_size
    batch_df = df.iloc[start:end]

    # Generate embeddings for the batch
    batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))

    rows_to_insert = batch_df.to_dict(orient='records')

    # Insert data into BigQuery
    errors = bq_client.insert_rows_json(table_id, rows_to_insert)

    if errors == []:
        df.loc[start:end, 'processed'] = True
        print(f"Batch from row {start} to {end} successfully inserted into BigQuery.")
    else:
        print(f"Errors occurred while inserting batch from row {start} to {end}:", errors)


/var/folders/df/gz0dgvjn2fsdflyvdqhzp4740000gp/T/ipykernel_47165/2918133766.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))


Batch from row 0 to 200 successfully inserted into BigQuery.


/var/folders/df/gz0dgvjn2fsdflyvdqhzp4740000gp/T/ipykernel_47165/2918133766.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))


Batch from row 200 to 400 successfully inserted into BigQuery.


/var/folders/df/gz0dgvjn2fsdflyvdqhzp4740000gp/T/ipykernel_47165/2918133766.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))


...

/var/folders/df/gz0dgvjn2fsdflyvdqhzp4740000gp/T/ipykernel_47165/2918133766.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))


Batch from row 5800 to 6000 successfully inserted into BigQuery.


/var/folders/df/gz0dgvjn2fsdflyvdqhzp4740000gp/T/ipykernel_47165/2918133766.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))


Batch from row 6000 to 6200 successfully inserted into BigQuery.


KeyboardInterrupt: 

In [49]:
import openai
import numpy as np
import pandas as pd
from google.cloud import bigquery
from concurrent.futures import ThreadPoolExecutor, as_completed

# Convert NumPy array to list of floats
def embedding_to_list(embedding):
    return embedding.tolist()

# Define BigQuery table ID
table_id = 'thermofigher-gen-ai.searchconsole.filtered_gsc_data_embedding'

# Function to process a batch of data
def process_batch(start, batch_df):
    batch_df['embedding'] = batch_df['query'].apply(lambda x: embedding_to_list(get_embedding(x)))
    rows_to_insert = batch_df.to_dict(orient='records')

    # Insert data into BigQuery
    errors = bq_client.insert_rows_json(table_id, rows_to_insert)
    return start, errors

# Process and insert data in batches of 200 rows
batch_size = 200
results = []

with ThreadPoolExecutor() as executor:
    futures = []
    for start in range(0, len(df), batch_size):
        end = start + batch_size
        batch_df = df.iloc[start:end].copy()

        # Submit the batch processing task to the executor
        futures.append(executor.submit(process_batch, start, batch_df))

    # Wait for all futures to complete and process the results
    for future in as_completed(futures):
        start, errors = future.result()
        if errors == []:
            # Mark the batch as processed
            df.loc[start:start + batch_size - 1, 'processed'] = True
            print(f"Batch from row {start} to {start + batch_size - 1} successfully inserted into BigQuery.")
        else:
            print(f"Errors occurred while inserting batch from row {start} to {start + batch_size - 1}:", errors)


Batch from row 2600 to 2799 successfully inserted into BigQuery.
Batch from row 1000 to 1199 successfully inserted into BigQuery.
Batch from row 1800 to 1999 successfully inserted into BigQuery.
Batch from row 1600 to 1799 successfully inserted into BigQuery.
Batch from row 600 to 799 successfully inserted into BigQuery.
Batch from row 2800 to 2999 successfully inserted into BigQuery.
Batch from row 2000 to 2199 successfully inserted into BigQuery.
Batch from row 1200 to 1399 successfully inserted into BigQuery.
Batch from row 1400 to 1599 successfully inserted into BigQuery.
Batch from row 800 to 999 successfully inserted into BigQuery.
Batch from row 3000 to 3199 successfully inserted into BigQuery.
Batch from row 3400 to 3599 successfully inserted into BigQuery.
Batch from row 0 to 199 successfully inserted into BigQuery.
Batch from row 400 to 599 successfully inserted into BigQuery.
Batch from row 2200 to 2399 successfully inserted into BigQuery.
Batch from row 3600 to 3799 success

In [44]:
df_unprocessed = df[df['processed'] != True]
df_unprocessed

,query,page,clicks,impressions,ctr,position,country,device,data_date,processed
6201,sulfuric acid concentrated msds,https://www.fishersci.com/msdsproxy%3FproductN...,20,135,0.315789,2.800000,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6202,conflikt,https://www.fishersci.com/shop/products/decon-...,20,85,0.269841,2.206349,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6203,pyridinium tribromide sds,https://www.fishersci.com/store/msds?partNumbe...,20,40,0.500000,2.000000,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6204,trans cinnamic acid sds,https://www.fishersci.com/store/msds?partNumbe...,20,52,0.400000,2.033333,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6205,fisher scientific msds,https://www.fishersci.com/us/en/customer-help-...,20,69,0.434783,2.200000,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
...,...,...,...,...,...,...,...,...,...,...
75320,dissecting forceps,https://www.fishersci.com/shop/products/fisher...,1,251,0.003984,4.410359,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
75321,ambion,https://www.fishersci.com/shop/products/ambion...,1,251,0.003984,8.733068,usa,MOBILE,2024-10-09 00:00:00+00:00,None
75322,periodoc table,https://www.fishersci.com/us/en/periodic-table...,1,252,0.003968,10.825397,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
75323,hydrion test strips,https://www.fishersci.com/shop/products/hydrio...,1,252,0.003968,9.837302,usa,MOBILE,2024-10-09 00:00:00+00:00,None


In [45]:
df = df_unprocessed.copy()
df

,query,page,clicks,impressions,ctr,position,country,device,data_date,processed
6201,sulfuric acid concentrated msds,https://www.fishersci.com/msdsproxy%3FproductN...,20,135,0.315789,2.800000,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6202,conflikt,https://www.fishersci.com/shop/products/decon-...,20,85,0.269841,2.206349,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6203,pyridinium tribromide sds,https://www.fishersci.com/store/msds?partNumbe...,20,40,0.500000,2.000000,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6204,trans cinnamic acid sds,https://www.fishersci.com/store/msds?partNumbe...,20,52,0.400000,2.033333,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
6205,fisher scientific msds,https://www.fishersci.com/us/en/customer-help-...,20,69,0.434783,2.200000,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
...,...,...,...,...,...,...,...,...,...,...
75320,dissecting forceps,https://www.fishersci.com/shop/products/fisher...,1,251,0.003984,4.410359,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
75321,ambion,https://www.fishersci.com/shop/products/ambion...,1,251,0.003984,8.733068,usa,MOBILE,2024-10-09 00:00:00+00:00,None
75322,periodoc table,https://www.fishersci.com/us/en/periodic-table...,1,252,0.003968,10.825397,usa,DESKTOP,2024-10-09 00:00:00+00:00,None
75323,hydrion test strips,https://www.fishersci.com/shop/products/hydrio...,1,252,0.003968,9.837302,usa,MOBILE,2024-10-09 00:00:00+00:00,None


In [ ]:

project_id = "thermofigher-gen-ai"
bq_client = bigquery.Client.from_service_account_json(project = project_id,json_credentials_path=cred)
query = """select * from thermofigher-gen-ai.searchconsole.filtered_gsc_data_embedding"""
df = bq_client.query(query).to_dataframe()
df

: 

In [56]:
df[0:50]

,query,page,clicks,impressions,ctr,position,country,device,data_date,embedding,processed
0,fisher scientific,https://www.fishersci.com/us/en/home.html,19612,66676,0.294139,1.013767,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[-0.005209560971707106, -0.01990083046257496, ...",<NA>
1,thermofisher,https://www.thermofisher.com/us/en/home.html,11486,28387,0.412552,1.001727,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[-0.019531551748514175, -0.022737380117177963,...",<NA>
2,fisher,https://www.fishersci.com/us/en/home.html,8621,46835,0.184072,1.128744,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[-0.0391501747071743, -0.020497869700193405, -...",<NA>
3,thermo fisher careers,https://jobs.thermofisher.com/global/en,4366,7807,0.649456,1.023694,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[-0.028729841113090515, 0.0022962053772062063,...",<NA>
4,thermo fisher careers,https://jobs.thermofisher.com/global/en,3746,6986,0.555530,1.009067,usa,MOBILE,2024-10-09 00:00:00+00:00,"[-0.028729841113090515, 0.0022962053772062063,...",<NA>
5,xnx,https://accelerator.geneai.thermofisher.com/io...,2849,41099,0.069320,1.356554,usa,MOBILE,2024-10-09 00:00:00+00:00,"[0.03747781738638878, -0.013691517524421215, 0...",<NA>
6,thermo fisher,https://www.thermofisher.com/us/en/home.html,2683,8586,0.321957,1.042860,usa,MOBILE,2024-10-09 00:00:00+00:00,"[-0.02688991278409958, -0.005080430302768946, ...",<NA>
7,acetone sds,https://www.fishersci.com/msds?productName=AC1...,2354,4828,0.707304,2.198413,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[0.012725039385259151, 0.018386904150247574, 0...",<NA>
8,iconnect thermofisher,https://iconnect.thermofisher.com/iconnect,1946,2466,0.826460,1.338873,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[-0.020499708130955696, -0.05275985226035118, ...",<NA>
9,interactive periodic table,https://www.fishersci.com/us/en/periodic-table...,1905,9562,0.216504,2.010405,usa,DESKTOP,2024-10-09 00:00:00+00:00,"[-0.056525420397520065, -0.028816338628530502,...",<NA>


In [ ]:
# import openai
import faiss
import numpy as np
import pandas as pd

def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

embeddings = np.array(df['embedding'].tolist()).astype('float32')
print(embeddings.shape)
print(embeddings[0])
print(df['embedding'][0])
print((df['embedding'][0]).shape)
normalized_embeddings = normalize_embeddings(embeddings)

# Set up FAISS index
dimension = normalized_embeddings.shape[1]
print("FAISS index dimension:", normalized_embeddings.shape)  # Debugging line

index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


def match_keyword(keyword, threshold=0.85):
    embedding = np.array(get_embedding(keyword)).astype('float32')
    normalized_embedding = normalize_embeddings(embedding.reshape(1, -1))
    
    # Perform the search
    distances, _ = index.search(normalized_embedding, k=1)
    
    # Convert L2 distance to cosine similarity
    l2_distance = distances[0][0]
    similarity = 1 - (l2_distance / 2)
    
    print(f"Cosine similarity: {similarity}")
    
    # Check against threshold
    return 1 if similarity >= threshold else None

In [76]:
match_keyword("adc therapy")

Cosine similarity: 0.6299470067024231


In [ ]:
faiss.write_index(index, "faiss_TF_index.bin")

In [68]:
loaded_index = faiss.read_index("faiss_TF_index.bin")

In [69]:
loaded_index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x30e749590> >

In [70]:
def match_keyword(keyword, threshold=0.85):
    embedding = np.array(get_embedding(keyword)).astype('float32')
    normalized_embedding = normalize_embeddings(embedding.reshape(1, -1))
    
    # Perform the search
    distances, _ = loaded_index.search(normalized_embedding, k=1)
    
    # Convert L2 distance to cosine similarity
    l2_distance = distances[0][0]
    similarity = 1 - (l2_distance / 2)
    
    print(f"Cosine similarity: {similarity}")
    
    # Check against threshold
    return 1 if similarity >= threshold else None